### Import Required Libraries and Set Up Environment Variables

In [9]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [10]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [12]:
# Set the base URL
base_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
request_url = f"{base_url}api-key={nyt_api_key}&fq={filter_query}&begin_date={begin_date}&end_date={end_date}&sort={sort}&fl={field_list}"

In [50]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    url = f'{request_url}&page={page}'
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews['response']['docs']:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f'Retrieved page number {page}.')
    except:
        # Print the page number that had no results then break from the loop
        print(f'No reviews retrieved for page number {page}.')
        break

Retrieved page number 0.
Retrieved page number 1.
Retrieved page number 2.
Retrieved page number 3.
Retrieved page number 4.
Retrieved page number 5.
Retrieved page number 6.
Retrieved page number 7.
Retrieved page number 8.
Retrieved page number 9.
Retrieved page number 10.
Retrieved page number 11.
Retrieved page number 12.
Retrieved page number 13.
Retrieved page number 14.
Retrieved page number 15.
Retrieved page number 16.
Retrieved page number 17.
Retrieved page number 18.
Retrieved page number 19.


In [98]:
# Make sure we retrieved 200 reviews
print(len(reviews_list))

# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

200
[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons

In [99]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_nyt_reviews = pd.json_normalize(reviews_list)

In [100]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
# df_nyt_reviews['title'] = df_nyt_reviews['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

# Using "\u2019 Review" to find the end of the title does not work too well because often the string in "headline.main" does not contain "\u2019 Review".
# For this reviews list using just "\u2019 " works better. That, of course is not foolproof either.
df_nyt_reviews['title'] = df_nyt_reviews['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 ")])

# Sometimes we end up with a trailing comma. Remove it.
df_nyt_reviews['title'] = df_nyt_reviews['title'].str.rstrip(',')

In [101]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_nyt_reviews['keywords'] = df_nyt_reviews['keywords'].apply(extract_keywords)

In [111]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = df_nyt_reviews['title'].to_list()

### Access The Movie Database API

In [179]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [182]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)
        print('Sleeping for 1 second.')

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    tmdb_response = requests.get(url + title + tmdb_key_string).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = tmdb_response['results'][0]['id']

        # Make a request for a the full movie details
        details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        tmdb_response_details = requests.get(details_url).json()
        
        # Extract the genre names into a list
        genres = []
        for i in range(len(tmdb_response_details['genres'])):
            genres.append(tmdb_response_details['genres'][i]['name'])

        # Extract the spoken_languages' English name into a list
        spoken_languages = []
        for i in range(len(tmdb_response_details['spoken_languages'])):
            spoken_languages.append(tmdb_response_details['spoken_languages'][i]['english_name'])

        # Extract the production_countries' name into a list
        production_countries = []
        for i in range(len(tmdb_response_details['production_countries'])):
            production_countries.append(tmdb_response_details['production_countries'][i]['name'])

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        dict_movie_details = {'title': title,
                              'original_title': tmdb_response_details['original_title'],
                              'budget': tmdb_response_details['budget'],
                              'original_language': tmdb_response_details['original_language'],
                              'homepage': tmdb_response_details['homepage'],
                              'overview': tmdb_response_details['overview'],
                              'popularity': tmdb_response_details['popularity'],
                              'runtime': tmdb_response_details['runtime'],
                              'revenue': tmdb_response_details['revenue'],
                              'release_date': tmdb_response_details['release_date'],
                              'vote_average': tmdb_response_details['vote_average'],
                              'vote_count': tmdb_response_details['vote_count'],
                              'genres': genres,
                              'spoken_languages': spoken_languages,
                              'production_countries': production_countries}
        tmdb_movies_list.append(dict_movie_details)
   
        # Print out the title that was found
        print(f'The movie called "{title}" was found.')
    except:
        print(f'The movie called "{title}" was not found.')


The movie called "The Attachment Diaries" was found.
The movie called "What’s Love Got to Do With It?" was found.
The movie called "You Can Live Forever" was found.
The movie called "A Tourist’s Guide to Love" was found.
The movie called "Other People’s Children" was found.
The movie called "One True Loves" was found.
The movie called "The Lost Weekend: A Love Story" was found.
The movie called "A Thousand and One" was found.
The movie called "Your Place or Mine" was found.
The movie called "Love in the Time of Fentanyl" was found.
The movie called "Pamela, a Love Story" was found.
The movie called "In From the Side" was found.
The movie called "After Love" was found.
The movie called "Alcarràs" was found.
The movie called "Nelly & Nadine" was found.
The movie called "Lady Chatterley’s Lover" was found.
The movie called "The Sound of Christmas" was found.
The movie called "The Inspection" was found.
The movie called "Bones and All" was found.
The movie called "My Policeman" was found.


In [183]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.601,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 3,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [184]:
# Convert the results to a DataFrame
df_tmdb_movies = pd.DataFrame(tmdb_movies_list)
df_tmdb_movies.head(10)

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.601,102,0,2021-10-07,3.000,3,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,en,,Two childhood friends now in their thirties mu...,24.836,109,10898395,2023-01-26,6.073,158,"[Romance, Comedy]","[English, Portuguese, Urdu]","[France, United Kingdom]"
2,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",64.137,96,15055,2023-03-24,6.600,31,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",19.584,96,0,2023-04-21,6.313,152,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",12.144,104,84178,2022-09-21,6.876,176,"[Drama, Comedy]","[English, French]",[France]
5,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,25.052,100,37820,2023-04-07,6.477,64,"[Romance, Drama, Comedy]","[English, Spanish]","[Czech Republic, United States of America]"
6,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,4.470,95,77145,2023-04-13,6.000,2,[Documentary],[English],[United States of America]
7,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,18.916,116,3395595,2023-03-31,6.911,79,"[Drama, Crime]","[English, Portuguese, Spanish]",[United States of America]
8,Your Place or Mine,Your Place or Mine,0,en,https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,35.033,109,0,2023-02-10,6.277,683,"[Romance, Comedy]",[English],[United States of America]
9,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,en,,An intimate portrait of a community fighting t...,2.513,85,0,2023-02-03,0.000,0,[Documentary],[English],"[Canada, United States of America]"


### Merge and Clean the Data for Export

In [212]:
# Merge the New York Times reviews and TMDB DataFrames on title
df_merged = pd.merge(df_tmdb_movies, df_nyt_reviews, on='title')

In [213]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
    df_merged[col] = df_merged[col].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        df_merged[col] = df_merged[col].str.replace(char, '')

# Display the fixed DataFrame
display(df_merged.head())

# Since the fixed columns are hidden in VSCode I am displaying them explicitly
df_merged[['genres', 'spoken_languages', 'production_countries']].head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.601,102,0,2021-10-07,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,en,,Two childhood friends now in their thirties mu...,24.836,109,10898395,2023-01-26,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",64.137,96,15055,2023-03-24,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",19.584,96,0,2023-04-21,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",12.144,104,84178,2022-09-21,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


,genres,spoken_languages,production_countries
0,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,"Romance, Comedy","English, Portuguese, Urdu","France, United Kingdom"
2,"Drama, Romance","English, French","Canada, United States of America"
3,"Romance, Comedy","English, Vietnamese",United States of America
4,"Drama, Comedy","English, French",France


In [214]:
# Drop "byline.person" column
df_merged.drop('byline.person', axis=1, inplace=True)

In [215]:
# Delete duplicate rows and reset index. Print shape before and after dropping duplicates
print(df_merged.shape)
df_merged.drop_duplicates(inplace=True)
print(df_merged.shape)
df_merged.reset_index(drop=True, inplace=True)

(201, 30)
(197, 30)


In [216]:
# Export data to CSV without the index
df_merged.to_csv('output/movie_data.csv', index=False)